# ACMA Scrapper
##### ©Haris Hassan

In [185]:
##=============================================================================
## ACMAscrapper V2.0.2
##=============================================================================
##
## Scrapper for Register of Radiocommunications Licences of Australian Communications and Media Authority
# 
#@Author Haris Hassan
#@Email haris.hassan@radhaz.com.au 
#@linkedin https://www.linkedin.com/in/hassanharis/
#
##=============================================================================
# Import libraries
import requests
import string
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option('display.max_colwidth', None)
#Reset index of dataframe after sorting


In [186]:
####Replace Site URL
site_url = 'https://web.acma.gov.au/pls/radcom/site_search.site_lookup?pSITE_ID=10030134'

In [187]:
ACMApage = requests.get(site_url)
soup = BeautifulSoup(ACMApage.text, 'lxml')

#Check if there's another Page
NextPageLink = ''.join(['https://web.acma.gov.au' + x for x in  [tl.get('href') for tl in soup.findAll('a',{'title':"Next Page"})]])

In [188]:
ACMAtable = soup.find('table',{"class": "tablelist responsive"})
ACMAheaders = [td.text.strip() for td in ACMAtable.select('th')]
ACMAdata = pd.DataFrame(columns = ACMAheaders)

ACMAlinks = []
for j in ACMAtable.find_all('tr')[1:]:
    ACMAValues = [tv.text.strip() for tv in j.find_all('td')]
    ACMAdata.loc[len(ACMAdata)] = ACMAValues
    ACMAlinks.append([tl.get('href') for tl in j.find_all('a')][0])
ACMAdata.insert(1,'links', ACMAlinks )

In [189]:
#Next Page
if NextPageLink and NextPageLink.strip():
    ACMApagep2 = requests.get(NextPageLink)
    soupp2 = BeautifulSoup(ACMApagep2.text, 'lxml')

    ACMAtablep2 = soupp2.find('table',{"class": "tablelist responsive"})
    ACMAheadersp2 = [td.text.strip() for td in ACMAtablep2.select('th')]
    ACMAdatap2 = pd.DataFrame(columns = ACMAheadersp2)

    ACMAlinksp2 = []
    for j in ACMAtablep2.find_all('tr')[1:]:
        ACMAValuesp2 = [tv.text.strip() for tv in j.find_all('td')]
        ACMAdatap2.loc[len(ACMAdatap2)] = ACMAValuesp2
        ACMAlinksp2.append([tl.get('href') for tl in j.find_all('a')][0])
    ACMAdatap2.insert(1,'links', ACMAlinksp2 )

    ACMAdata_final = pd.concat([ACMAdata, ACMAdatap2], axis=0)
else:
    ACMAdata_final = ACMAdata
#Reset index of dataframe after sorting
ACMAdata_final = ACMAdata_final.reset_index(drop=True)

In [190]:
#removing duplicates
ACMAdatalinks = list( dict.fromkeys(ACMAdata_final['links']) )
display(ACMAdata_final)

,ID,links,Frequency,Emission Designator,Azimuth,T/R,Client,BSL/Licence No
0,11702112,/rrl/assignment_search.lookup?pDEVICE_REGISTRATION_ID=11702112,1.8575 GHz,15M0W7D,85°,T,Optus Mobile Pty Limited (1103276),9263448
1,11702113,/rrl/assignment_search.lookup?pDEVICE_REGISTRATION_ID=11702113,1.8575 GHz,15M0W7D,285°,T,Optus Mobile Pty Limited (1103276),9263448
2,11702116,/rrl/assignment_search.lookup?pDEVICE_REGISTRATION_ID=11702116,1.7625 GHz,15M0W7D,85°,R,Optus Mobile Pty Limited (1103276),9263448
3,11702117,/rrl/assignment_search.lookup?pDEVICE_REGISTRATION_ID=11702117,1.7625 GHz,15M0W7D,285°,R,Optus Mobile Pty Limited (1103276),9263448
4,11701991,/rrl/assignment_search.lookup?pDEVICE_REGISTRATION_ID=11701991,708 MHz,10M0W7D,85°,R,Optus Mobile Pty Limited (1149289),9469858
5,11701992,/rrl/assignment_search.lookup?pDEVICE_REGISTRATION_ID=11701992,708 MHz,10M0W7D,285°,R,Optus Mobile Pty Limited (1149289),9469858
6,11701970,/rrl/assignment_search.lookup?pDEVICE_REGISTRATION_ID=11701970,763 MHz,10M0W7D,85°,T,Optus Mobile Pty Limited (1149289),9469858
7,11701971,/rrl/assignment_search.lookup?pDEVICE_REGISTRATION_ID=11701971,763 MHz,10M0W7D,285°,T,Optus Mobile Pty Limited (1149289),9469858
8,11701912,/rrl/assignment_search.lookup?pDEVICE_REGISTRATION_ID=11701912,2.14 GHz,20M0W7D,85°,T,Optus Mobile Pty Limited (1103276),10143562
9,11701916,/rrl/assignment_search.lookup?pDEVICE_REGISTRATION_ID=11701916,1.95 GHz,20M0W7D,85°,R,Optus Mobile Pty Limited (1103276),10143562


In [191]:
AntennaTableHeader = ['Licence Number', 'Antenna', 'Client','Device Type','Emission Center Frequency','Transmitter Power',
                      'Antenna Height (AGL)', 
                      'Antenna Polarisation','Antenna Azimuth', 'Antenna Tilt','Device Registration ID','Date Authorised']
AntennaTable = pd.DataFrame(columns = AntennaTableHeader)


for acmalink in ACMAdatalinks:
    url = 'https://web.acma.gov.au'+ acmalink
    page = requests.get(url)
    soup2 = BeautifulSoup(page.text, 'lxml')
    
    try:
        Antennaheaders = [td.text for td in soup2.select_one('table',{"class": "tabledetail"}).select('td:nth-of-type(1)') if not td.has_attr('colspan')]
        AntennaValues = [td.text.strip() for td in soup2.select_one('table',{"class": "tabledetail"}).select('td:nth-of-type(2)')]
    except Exception as e:
        print (e)
        continue
        
    if url[-2:]=='/1':
        try:
            Antennaheaders2 = [td.text for td in soup2.select_one('table:nth-of-type(2)',{"class": "tabledetail"}).select('td:nth-of-type(1)') if not td.has_attr('colspan')]
            AntennaValues2 = [td.text.strip() for td in soup2.select_one('table:nth-of-type(2)',{"class": "tabledetail"}).select('td:nth-of-type(2)')]
        except Exception as e:
            print(e)
            print(acmalink)
            continue
    
    AntennaDictionary = {Antennaheaders[i]: AntennaValues[i] for i in range(len(Antennaheaders))}
    AntennaTable_toAdd = pd.DataFrame([AntennaDictionary])
    AntennaTable = pd.concat([AntennaTable, AntennaTable_toAdd], ignore_index=True)
    
    if url[-2:]=='/1':
        AntennaTable_toAdd2 = pd.DataFrame([{Antennaheaders2[i]: AntennaValues2[i] for i in range(len(Antennaheaders))}])
        AntennaTable = pd.concat([AntennaTable, AntennaTable_toAdd2], ignore_index=True)

In [192]:
#Formatting Data
AntennaTable = AntennaTable.fillna('')
AntennaTable['Antenna']=([''.join(x[2].title() +' '+ x[1] + ' '+ x[0].title()) for x in AntennaTable['Antenna'].str.split(',', 2)])


if 'EFL ID' in AntennaTable:
    AntennaTable['Device Registration ID'] = AntennaTable['Device Registration ID'].astype(str) + AntennaTable['EFL ID']

    
AntennaTable['Antenna']= AntennaTable['Antenna'].str.strip()
AntennaTable['Client']=AntennaTable['Client'].apply(lambda x: x.title())
AntennaTable['Client']=AntennaTable['Client'].str.replace('Limited', 'Ltd')



In [193]:


Renamed_headers = ['Licence', 'Antenna', 'Client','Type','Frequency','Power','Height', 
                      'Polarisation','Azimuth', 'Tilt','Device ID','Date Authorised']

AntennaTable_export = AntennaTable
AntennaTable_export = AntennaTable_export.rename(columns={'Licence Number': 'Licence', 'Device Type': 'Type', 
                                                          'Emission Center Frequency': 'Frequency'
                                                          ,'Transmitter Power': 'Power'
                                                          , 'Antenna Height (AGL)': 'Height', 
                                                          'Antenna Polarisation': 'Polarisation', 
                                                          'Antenna Azimuth': 'Azimuth', 
                   'Device Registration ID': 'Device ID','Antenna Tilt': 'Tilt'})

#AntennaTable_export = AntennaTable_export.rename(columns= dict(zip(Antennaheaders, Renamed_headers)))

AntennaTable_export = AntennaTable_export.reset_index(drop=True)
AntennaTable_export.index += 1
display(AntennaTable_export[Renamed_headers])


,Licence,Antenna,Client,Type,Frequency,Power,Height,Polarisation,Azimuth,Tilt,Device ID,Date Authorised
1,9263448,Tongyu T1001W6F022_1.8_2.1 Panel(1Sector),Optus Mobile Pty Ltd,Transmitter,1.8575 GHz,240 W pY,6.4 m,Slant,85°,0°,11702112,26/Jun/2022
2,9263448,Tongyu T1001W6F022_1.8_2.1 Panel(1Sector),Optus Mobile Pty Ltd,Transmitter,1.8575 GHz,240 W pY,6.4 m,Slant,285°,0°,11702113,26/Jun/2022
3,9263448,Tongyu T1001W6F022_1.8_2.1 Panel(1Sector),Optus Mobile Pty Ltd,Receiver,1.7625 GHz,,6.4 m,Slant,85°,0°,11702116,26/Jun/2022
4,9263448,Tongyu T1001W6F022_1.8_2.1 Panel(1Sector),Optus Mobile Pty Ltd,Receiver,1.7625 GHz,,6.4 m,Slant,285°,0°,11702117,26/Jun/2022
5,9469858,Tongyu T1001W6F022_0.7 Panel(1Sector),Optus Mobile Pty Ltd,Receiver,708 MHz,,6.4 m,Slant,85°,0°,11701991,26/Jun/2022
6,9469858,Tongyu T1001W6F022_0.7 Panel(1Sector),Optus Mobile Pty Ltd,Receiver,708 MHz,,6.4 m,Slant,285°,0°,11701992,26/Jun/2022
7,9469858,Tongyu T1001W6F022_0.7 Panel(1Sector),Optus Mobile Pty Ltd,Transmitter,763 MHz,320 W pY,6.4 m,Slant,85°,0°,11701970,26/Jun/2022
8,9469858,Tongyu T1001W6F022_0.7 Panel(1Sector),Optus Mobile Pty Ltd,Transmitter,763 MHz,320 W pY,6.4 m,Slant,285°,0°,11701971,26/Jun/2022
9,10143562,Tongyu T1001W6F022_1.8_2.1 Panel(1Sector),Optus Mobile Pty Ltd,Transmitter,2.14 GHz,280 W pY,6.4 m,Slant,85°,0°,11701912,26/Jun/2022
10,10143562,Tongyu T1001W6F022_1.8_2.1 Panel(1Sector),Optus Mobile Pty Ltd,Receiver,1.95 GHz,,6.4 m,Slant,85°,0°,11701916,26/Jun/2022


In [194]:
AntennaTable_export.to_excel(r'C:\Users\Mewtwo\Desktop'+'\\'+ AntennaDictionary['Site'] + '.xlsx', columns = Renamed_headers, index=True)

In [146]:
AntennaTable_export.to_html(r'C:\Users\Mewtwo\Desktop\Antennadata.html', columns = Renamed_headers, index=True)
import subprocess
subprocess.call('wkhtmltoimage -f png --width 0 Antennadata.html Antennadata.png', shell=True)



1

In [9]:
%notebook "C:\Users\Mewtwo\Documents\Python\ACMAscrapper-Version 2.ipynb"